In [1]:
library(lme4)
library(simr)
library(dplyr)
library(broom)
install.packages("sjPlot")
library(sjPlot)
install.packages("broom.mixed")
library(broom.mixed)
library(htmlTable)
library(dplyr)
library(stringr)
library(car)
install.packages("stargazer")
library(stargazer)
install.packages("lmerTest")
library(lmerTest)

Loading required package: Matrix


Attaching package: ‘simr’


The following object is masked from ‘package:lme4’:

    getData



Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union





The downloaded binary packages are in
	/var/folders/h_/pm5z54tn7rj170br8494ljz80000gn/T//RtmpTHvWWt/downloaded_packages


Learn more about sjPlot with 'browseVignettes("sjPlot")'.




The downloaded binary packages are in
	/var/folders/h_/pm5z54tn7rj170br8494ljz80000gn/T//RtmpTHvWWt/downloaded_packages



Attaching package: ‘stringr’


The following object is masked from ‘package:simr’:

    fixed


Loading required package: carData


Attaching package: ‘car’


The following object is masked from ‘package:dplyr’:

    recode





The downloaded binary packages are in
	/var/folders/h_/pm5z54tn7rj170br8494ljz80000gn/T//RtmpTHvWWt/downloaded_packages



Please cite as: 


 Hlavac, Marek (2022). stargazer: Well-Formatted Regression and Summary Statistics Tables.

 R package version 5.2.3. https://CRAN.R-project.org/package=stargazer 





The downloaded binary packages are in
	/var/folders/h_/pm5z54tn7rj170br8494ljz80000gn/T//RtmpTHvWWt/downloaded_packages



Attaching package: ‘lmerTest’


The following object is masked from ‘package:lme4’:

    lmer


The following object is masked from ‘package:stats’:

    step




In [2]:

# file_path_total <- 'stats_ltm_distractor/pilot_total/total/df_v2_rel_neg.csv'
# file_path_5 <- 'stats_ltm_distractor/pilot5/total/df_v2_rel_neg.csv'
# file_path_6 <- 'stats_ltm_distractor/pilot6/total/df_v2_rel_neg.csv'

# file_path_total <- 'stats_ltm_distractor/pilot_total/total/df_v2_rel_pos.csv'
# file_path_5 <- 'stats_ltm_distractor/pilot5/total/df_v2_rel_pos.csv'
# file_path_6 <- 'stats_ltm_distractor/pilot6/total/df_v2_rel_pos.csv'


# file_path_total <- 'stats_ltm_distractor/pilot_total/total/df_it_rel_neg.csv'
# file_path_5 <- 'stats_ltm_distractor/pilot5/total/df_it_rel_neg.csv'
# file_path_6 <- 'stats_ltm_distractor/pilot6/total/df_it_rel_neg.csv'

file_path_total <- 'stats_ltm_distractor/pilot_total/total/df_it_rel_pos.csv'
file_path_5 <- 'stats_ltm_distractor/pilot5/total/df_it_rel_pos.csv'
file_path_6 <- 'stats_ltm_distractor/pilot6/total/df_it_rel_pos.csv'



df_total <- read.csv(file_path_total) 
df5 <- read.csv(file_path_5) 
df6 <- read.csv(file_path_6) 


In [3]:
control <- lmerControl(optimizer = "bobyqa")#, optCtrl = list(maxfun = 10000))

In [4]:
run_ltm_models <- function(df) {
    
    ltm_rel <- glmer(recall_answ ~ it_sim_dis_diff_test_z + it_sim_dis_diff_test_sq_z + it_sim_dis_diff_test_cube_z  
                       + v2_sim_dis_diff_test_z + v2_sim_dis_diff_test_sq_z + v2_sim_dis_diff_test_cube_z  
                       + reliability_binary_z *  validity_binary_z + (1 | participant), 
                      data = df, family = binomial,control = glmerControl(optimizer = "bobyqa",
                                            optCtrl = list(maxfun = 500000),
                                            calc.derivs = FALSE))

return(list( 
    ltm_rel  = ltm_rel
  ))
}



In [5]:
results_old_stims <- run_ltm_models(df6)
results_new_stims <- run_ltm_models(df5)
results_total <- run_ltm_models(df_total)




In [6]:
library(htmlTable)

extract_significance <- function(model_summary) {
  coef_table <- model_summary$coefficients
  var_names <- rownames(coef_table)
  
  # Get p-values
  if ("Pr(>|z|)" %in% colnames(coef_table)) {
    p_values <- coef_table[, "Pr(>|z|)"]
  } else if ("Pr(>|t|)" %in% colnames(coef_table)) {
    p_values <- coef_table[, "Pr(>|t|)"]
  } else {
    stop("Cannot find p-values in coefficients table")
  }
  
  # Calculate significance stars
  significance <- ifelse(is.na(p_values), "",
                        ifelse(p_values < 0.001, "***",
                              ifelse(p_values < 0.01, "**",
                                    ifelse(p_values < 0.05, "*",
                                          ifelse(p_values < 0.1, ".", "")))))
  
  return(data.frame(
    variable = var_names,
    p_value = p_values,
    significance = significance,
    stringsAsFactors = FALSE
  ))
}

compare_models <- function(model_list, model_names, 
                          include_p_values = TRUE,
                          only_significant = TRUE) {
  
  # Extract significance for all models
  sig_list <- lapply(model_list, extract_significance)
  names(sig_list) <- model_names
  
  # Get all variables
  all_vars <- unique(unlist(lapply(sig_list, function(x) x$variable)))
  
  # Create result dataframe
  result_df <- data.frame(Variable = all_vars, stringsAsFactors = FALSE)
  
  # Add columns for each model
  for (model_name in model_names) {
    model_sig <- sig_list[[model_name]]
    
    result_df[[model_name]] <- sapply(all_vars, function(var) {
      idx <- which(model_sig$variable == var)
      if (length(idx) == 0) return("")
      
      p_val <- model_sig$p_value[idx]
      stars <- model_sig$significance[idx]
      
      if (is.na(p_val)) return("")
      
      if (include_p_values) {
        if (stars != "") {
          return(paste0(stars, "<br>", sprintf("%.4f", p_val)))
        } else {
          return(sprintf("%.4f", p_val))
        }
      } else {
        return(stars)
      }
    })
  }
  
  # Filter to only significant if requested
  if (only_significant) {
    # Keep rows where at least one model has significance
    has_sig <- apply(result_df[, -1, drop = FALSE], 1, function(row) {
      any(grepl("[*.]", row))
    })
    result_df <- result_df[has_sig, , drop = FALSE]
  }
  
  # Remove completely empty rows
  non_empty <- apply(result_df[, -1, drop = FALSE], 1, function(row) any(row != ""))
  result_df <- result_df[non_empty, , drop = FALSE]
  
  # Create HTML table
  html_table <- htmlTable(
    result_df,
    caption = "Model Comparison<br><small>*** p&lt;0.001, ** p&lt;0.01, * p&lt;0.05, . p&lt;0.1</small>",
    rnames = FALSE,
    css.cell = "padding: 6px; border: 1px solid #ddd;",
    css.table = "border-collapse: collapse; width: 100%; font-family: Arial, sans-serif;",
    css.header = "background-color: #f2f2f2; font-weight: bold;"
  )
  
  return(html_table)
}


In [7]:
ltm_rel_old <- summary(results_old_stims$ltm_rel)
ltm_rel_new <- summary(results_new_stims$ltm_rel)
ltm_rel_total <- summary(results_total$ltm_rel)

comparison_table_rel <- compare_models(
  model_list = list(ltm_rel_old, ltm_rel_new, ltm_rel_total),
  model_names = c("Old Stimuli", "New Stimuli", "Total"),
  include_p_values = TRUE,
  only_significant = TRUE
)

IRdisplay::display_html(comparison_table_rel)


Model Comparison *** p<0.001, ** p<0.01, * p<0.05, . p<0.1 
 
 Variable 
 Old Stimuli 
 New Stimuli 
 Total 
 
 
 
 
 (Intercept) 
 *** 0.0002 
 ** 0.0029 
 *** 0.0000 
 
 
 it_sim_dis_diff_test_z 
 0.5820 
 ** 0.0045 
 . 0.0898 
 
 
 it_sim_dis_diff_test_sq_z 
 *** 0.0004 
 0.5376 
 ** 0.0065 
 
 
 it_sim_dis_diff_test_cube_z 
 0.1940 
 0.6176 
 0.4450 
 
 
 v2_sim_dis_diff_test_z 
 0.4498 
 *** 0.0002 
 * 0.0442 
 
 
 v2_sim_dis_diff_test_sq_z 
 * 0.0129 
 ** 0.0029 
 *** 0.0000 
 
 
 v2_sim_dis_diff_test_cube_z 
 0.9431 
 *** 0.0003 
 ** 0.0071 
 
 
 reliability_binary_z 
 0.4492 
 0.4675 
 0.4179 
 
 
 validity_binary_z 
 0.6255 
 0.5129 
 0.9826 
 
 
 reliability_binary_z:validity_binary_z 
 0.9371 
 0.8474 
 0.6975

In [8]:
summary(results_old_stims$ltm_rel)

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: recall_answ ~ it_sim_dis_diff_test_z + it_sim_dis_diff_test_sq_z +  
    it_sim_dis_diff_test_cube_z + v2_sim_dis_diff_test_z + v2_sim_dis_diff_test_sq_z +  
    v2_sim_dis_diff_test_cube_z + reliability_binary_z * validity_binary_z +  
    (1 | participant)
   Data: df
Control: glmerControl(optimizer = "bobyqa", optCtrl = list(maxfun = 5e+05),  
    calc.derivs = FALSE)

     AIC      BIC   logLik deviance df.resid 
  2584.5   2645.8  -1281.2   2562.5     1933 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-2.2138 -0.8046 -0.5670  0.9642  2.2249 

Random effects:
 Groups      Name        Variance Std.Dev.
 participant (Intercept) 0.4436   0.666   
Number of obs: 1944, groups:  participant, 134

Fixed effects:
                                        Estimate Std. Error z value Pr(>|z|)
(Intercept)                            -0.279893   0.076